## Invitae Bfx Exercise B.3

In [1]:
import pandas as pd
import re

#### The complete set of scripts will include the following:
#### 1. README file
#### 2. src folder with .py scripts
#### 3. testing example
#### 4. ipython notebook to record process of thinking, explain possible strenght and weakness

Design

variables:
1. hang: Input_file_1[2]
2. tr_coord: Input_file_2[1]
3. tr_remain: the remainig int in trascript that is yet aligned
4. genome_output: the output in output_file[3]

logic:
1. the starting tr_remain is Input_file_2[1]
   the starting genome_output is "hang"
2. when M: 
    genome_output = hang + min(value_M, tr_remain)
    tr_remain = tr_remain - min(tr_remain, value_M)
   when D:
    genome_output + = value_D
   when I:
    tr_remain = tr_remain - (value_I)
    genome_output remains
 

### Explore the CIGAR string conversion function

In [2]:
## CIGAR string parsing

import re

CIGAR = "8M7D6M2I2M11D7M"

match = re.findall(r'(\d+)(\w)', CIGAR) # turn the CIGAR string into a list
print (match)

[('8', 'M'), ('7', 'D'), ('6', 'M'), ('2', 'I'), ('2', 'M'), ('11', 'D'), ('7', 'M')]


In [3]:
for pair in match:
    print("int", pair[0], "Symbol", pair[1])

int 8 Symbol M
int 7 Symbol D
int 6 Symbol M
int 2 Symbol I
int 2 Symbol M
int 11 Symbol D
int 7 Symbol M


### The main functions to convert the transcript coordinate to genome coordinate

In the query, what if the transcript coordiante is too large?
When the coord is larger than the total length of the cigar string (sum of M + I), need to return something to trigger error message

Test case: tr_coord = "100"

In [4]:
# hang: Input_file_1[2]
# cigar = Input_file_1[3]
# tr_coord = Input_file_2[1]

import pandas as pd
import re

def convert_cigar_string_to_list(cigar):
    cigar_list = re.findall(r'(\d+)(\w)', cigar)
    return cigar_list

def transcript_coord_to_genome_coord(hang, cigar, tr_coord):
    genome_output = int(hang) - 1 # accomodate for 0-based index
    tr_remain = int(tr_coord) + 1 # accomodate for 0-based index
    
    cigar_list = convert_cigar_string_to_list(cigar)
    for value, cigar_type in cigar_list:
        if int(tr_remain) == 0:
            break
        if cigar_type == "M":
            genome_output += min(int(value), tr_remain)
            tr_remain -= min(tr_remain, int(value))
        elif cigar_type == "D":
            genome_output += int(value)
        elif cigar_type == "I":
            tr_remain -= min(tr_remain, int(value))
    print("tr_remain", tr_remain)
    # when the tr_remain is not eventually 0, then "tr_coord" is too large, impossible to find matching genome coordinate
    if tr_remain > 0:
        return -1
  
    return str(genome_output)

In [5]:
### Example genome and transcript sequences
genome_chr1 = "ACTGTCATGTACGTTTAGCTAGCCTAGCTAGGGACCTAGATAATTTAGCTAG"
tr_chr1 = "GTCATGTACTAGCCGGTAAGATAAT"

### Tested several cases for the function "transcript_coord_to_genome_coord", now read query line by line

In [6]:
### When the tr_coord is too large
hang = "3"
cigar = "8M7D6M2I2M11D7M"
tr_coord = "100"

result = transcript_coord_to_genome_coord(hang, cigar, tr_coord)
print(result)
#print("genome_output", result, "tr_nt", tr_chr1[int(tr_coord)], "genome_nt", genome_chr1[int(result)])

tr_remain 76
-1


In [7]:
### when the tr_coord is the first nucleotide
hang = "3"
cigar = "8M7D6M2I2M11D7M"
tr_coord = "0"

result = transcript_coord_to_genome_coord(hang, cigar, tr_coord)
print("genome_output", result, "tr_nt", tr_chr1[int(tr_coord)], "genome_nt", genome_chr1[int(result)])

tr_remain 0
genome_output 3 tr_nt G genome_nt G


In [8]:
### when the tr_coord is in the first "M" part
hang = "3"
cigar = "8M7D6M2I2M11D7M"
tr_coord = "4"

result = transcript_coord_to_genome_coord(hang, cigar, tr_coord)
print("genome_output", result, "tr_nt", tr_chr1[int(tr_coord)], "genome_nt", genome_chr1[int(result)])

tr_remain 0
genome_output 7 tr_nt T genome_nt T


In [9]:
### when the tr_coord is the last nt of the "M" part
hang = "3"
cigar = "8M7D6M2I2M11D7M"
tr_coord = "7"

result = transcript_coord_to_genome_coord(hang, cigar, tr_coord)
print("genome_output", result, "tr_nt", tr_chr1[int(tr_coord)], "genome_nt", genome_chr1[int(result)])

tr_remain 0
genome_output 10 tr_nt A genome_nt A


In [10]:
### when the tr_coord is the first nt in "M" after a "D"
hang = "3"
cigar = "8M7D6M2I2M11D7M"
tr_coord = "8"

result = transcript_coord_to_genome_coord(hang, cigar, tr_coord)
print("genome_output", result, "tr_nt", tr_chr1[int(tr_coord)], "genome_nt", genome_chr1[int(result)])

tr_remain 0
genome_output 18 tr_nt C genome_nt C


In [11]:
### when the tr_coord is the second nt in "M" after a "D"
hang = "3"
cigar = "8M7D6M2I2M11D7M"
tr_coord = "9"

result = transcript_coord_to_genome_coord(hang, cigar, tr_coord)
print("genome_output", result, "tr_nt", tr_chr1[int(tr_coord)], "genome_nt", genome_chr1[int(result)])

tr_remain 0
genome_output 19 tr_nt T genome_nt T


In [12]:
### when the tr_coord is the last nt in "M" before "I"
hang = "3"
cigar = "8M7D6M2I2M11D7M"
tr_coord = "13"

result = transcript_coord_to_genome_coord(hang, cigar, tr_coord)
print("genome_output", result, "tr_nt", tr_chr1[int(tr_coord)], "genome_nt", genome_chr1[int(result)])

tr_remain 0
genome_output 23 tr_nt C genome_nt C


In [13]:
### when the tr_coord is the first nt in "I"
hang = "3"
cigar = "8M7D6M2I2M11D7M"
tr_coord = "14"

result = transcript_coord_to_genome_coord(hang, cigar, tr_coord)
print("genome_output", result, "tr_nt", tr_chr1[int(tr_coord)], "genome_nt", genome_chr1[int(result)])

tr_remain 0
genome_output 23 tr_nt G genome_nt C


In [14]:
### when the tr_coord is the last nt in "I"
hang = "3"
cigar = "8M7D6M2I2M11D7M"
tr_coord = "15"

result = transcript_coord_to_genome_coord(hang, cigar, tr_coord)
print("genome_output", result, "tr_nt", tr_chr1[int(tr_coord)], "genome_nt", genome_chr1[int(result)])

tr_remain 0
genome_output 23 tr_nt G genome_nt C


In [15]:
### when the tr_coord is the first nt in "M" after the "I"
hang = "3"
cigar = "8M7D6M2I2M11D7M"
tr_coord = "16"

result = transcript_coord_to_genome_coord(hang, cigar, tr_coord)
print("genome_output", result, "tr_nt", tr_chr1[int(tr_coord)], "genome_nt", genome_chr1[int(result)])

tr_remain 0
genome_output 24 tr_nt T genome_nt T


In [16]:
### when the tr_coord is the last nt in "M" before "D"
hang = "3"
cigar = "8M7D6M2I2M11D7M"
tr_coord = "17"

result = transcript_coord_to_genome_coord(hang, cigar, tr_coord)
print("genome_output", result, "tr_nt", tr_chr1[int(tr_coord)], "genome_nt", genome_chr1[int(result)])

tr_remain 0
genome_output 25 tr_nt A genome_nt A


In [17]:
### when the tr_coord is the first nt in "M" after "D" 
hang = "3"
cigar = "8M7D6M2I2M11D7M"
tr_coord = "18"

result = transcript_coord_to_genome_coord(hang, cigar, tr_coord)
print("genome_output", result, "tr_nt", tr_chr1[int(tr_coord)], "genome_nt", genome_chr1[int(result)])

tr_remain 0
genome_output 37 tr_nt A genome_nt A


In [18]:
### when the tr_coord is the last index of the transcript
hang = "3"
cigar = "8M7D6M2I2M11D7M"
tr_coord = "24"

result = transcript_coord_to_genome_coord(hang, cigar, tr_coord)
print("genome_output", result, "tr_nt", tr_chr1[int(tr_coord)], "genome_nt", genome_chr1[int(result)])

tr_remain 0
genome_output 43 tr_nt T genome_nt T


In [19]:
### when the tr_coord is 1 nt out of the CIGAR
hang = "3"
cigar = "8M7D6M2I2M11D7M"
tr_coord = "25"

result = transcript_coord_to_genome_coord(hang, cigar, tr_coord)
print("genome_output", result)
#print("genome_output", result, "tr_nt", tr_chr1[int(tr_coord)], "genome_nt", genome_chr1[int(result)])

tr_remain 1
genome_output -1


### Try read-in and read-out files

In [20]:
Input_file_1 = pd.read_csv("/Users/huangz36/Documents/invitae_DC/example/input_file_1.txt", sep="\t", header = None, index_col=0)
print(Input_file_1.head())

        1   2                3
0                             
TR1  CHR1   3  8M7D6M2I2M11D7M
TR2  CHR2  10              20M


In [21]:
trans_ID = "TR1"

TR_chr, cigar = Input_file_1.loc[trans_ID, 1], Input_file_1.loc[trans_ID, 3]

print(TR_chr, cigar)

CHR1 8M7D6M2I2M11D7M


In [22]:
output_file = open("/Users/huangz36/Documents/invitae_DC/example/output_file.txt", "w")

## Save Input_file_1 as a DF for easy searching and matching, each TR is unique
## Read Input_file_2 line by line
Input_file_1 = pd.read_csv("/Users/huangz36/Documents/invitae_DC/example/input_file_1.txt", sep="\t", header = None, index_col=0)
Input_file_2 = open("/Users/huangz36/Documents/invitae_DC/example/input_file_2.txt", "r")
for line in Input_file_2:
    trans_ID, tr_coord = line.strip().split("\t")[0], line.strip().split("\t")[1]
    tr_chr, hang, cigar = Input_file_1.loc[trans_ID, 1], Input_file_1.loc[trans_ID, 2], Input_file_1.loc[trans_ID, 3]
    
    genome_coord = transcript_coord_to_genome_coord(hang, cigar, tr_coord)
    L = [trans_ID, tr_coord, tr_chr, genome_coord]
    print('\t'.join(L))
    output_file.write('\t'.join(L))
    output_file.write('\n')
    
Input_file_2.close()
output_file.close()

tr_remain 0
TR1	4	CHR1	7
tr_remain 0
TR2	0	CHR2	10
tr_remain 0
TR1	13	CHR1	23
tr_remain 0
TR2	10	CHR2	20


### Construct argparse

In [23]:
#!/usr/bin/env python3

import argparse
import logging
import os
import sys
import pandas as pd
import re

def make_arg_parser():
    parser = argparse.ArgumentParser(description='python3 trans_to_genome.py ' \
                                 '--input_file_1_path path to input file_1.txt' \
                                 '--input_file_2_path path to input file_2.txt' \
                                 '--output_file_path path to output file')
    parser.add_argument("--input_file_1_path", required=True)
    parser.add_argument("--input_file_2_path", required=True)
    parser.add_argument("--output_file_path", required=True)
    return parser


def main(argv):
    # set up the parser
    parser = make_arg_parser()
    args = parser.parse_args()
    if not args.input_file_1_path:
        logging.error("---- invalid input_file_1")
    elif not os.path.isfile(args.input_file_1_path):
        logging.error("---- empty input file 1")
    else:
        input_file_1_path = args.input_file_1_path 
    if not args.input_file_2_path:
        logging.error("---- invalid input_file_2")
    elif not os.path.isfile(args.input_file_2_path): 
        logging.error("---- empty input file 2")
    else:
        input_file_2_path = input_file_2_path
    if not args.output_file_path:
        logging.error("--- invalid output file name")
    else:
        output_file_path = args.output_file_path
    # set up the log file
    output_dir = "/".join(output_file_path.split("/")[0:-1])
    log_file_name = '{}/transcript_to_genomics_coord.log'.format(output_dir)
    logging.basicConfig(filename=log_file_name, level = logging.INFO, format = '%(asctime)s:%(name)s:%(levelname)s:%(message)s')
    logger = logging.getLogger('trans_to_genome')
    if os.path.exists(log_file_name):
        os.remove(log_file_name)
    # set up output file
    output_file = open(output_file_path, "w")
    # read input_file_1 as a pandas df
    input_file_1 = pd.read_csv(input_file_1_path, sep="\t", header = None, index_col=0)
    # read input_file_2 line by line, calculate genomic coord, and write to output_file line by line 
    input_file_2 = open(input_file_2_path, "r")
    for line in Input_file_2:
        trans_ID, tr_coord = line.strip().split("\t")[0], line.strip().split("\t")[1]
        tr_chr, hang, cigar = input_file_1.loc[trans_ID, 1], input_file_1.loc[trans_ID, 2], input_file_1.loc[trans_ID, 3]
        genome_coord = transcript_coord_to_genome_coord(hang, cigar, tr_coord)
        L = [trans_ID, tr_coord, tr_chr, genome_coord]
        print('\t'.join(L))
        output_file.write('\t'.join(L))
        output_file.write('\n')
    input_file_2.close()
    output_file.close()
    logger.info("--- the conversion is complete")

if __name__ == "__main__":
        main(sys.argv)

usage: ipykernel_launcher.py [-h] --input_file_1_path INPUT_FILE_1_PATH
                             --input_file_2_path INPUT_FILE_2_PATH
                             --output_file_path OUTPUT_FILE_PATH
ipykernel_launcher.py: error: the following arguments are required: --input_file_1_path, --input_file_2_path, --output_file_path


SystemExit: 2

/Users/huangz36/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2870: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## Other things to test and consider

1. State any assumptions made
  - Assume coordinate in transcript query always valid, cooridnate <= (M+I). If invalid, do not stop the execution, print out the genomic coordinate as "-1" instead
2. Document key strength / weakness of the implementation
  - The input_file_1 is read as a dataframe, by making the assumption that the transcript table is accessed multiple times, given a long query list
  - When the input_file_1 is extremely big, need a more efficient way to search for the correct row with chromosome, hang and cigar information
  - Memory efficiency is low when we need to use dataframe to save the input_file_1, use "startwith"
3. Consider: 
  - Input_file_1 short, input_file_2 long (the current method, save the input_file_1 into a dataframe).
  - Input_file_1 long, input_file_2 is short (use startwith, to scan through the transcript information sheet each query, and use no memory)
4. Function level test cases please see the ipython notebook. Input level errors handled by the logging (eg. empty input, non-existing files)
  - What if the transcript is impossible? (as long as cigar and genome mapping (0-based starting position) possible, col3 cannot be negative, col4 cannot contain illegal characters)
    - Check column number
    - Check content of column 3 and column 4
  - What if the query is impossible
    - The number is larger than the total length of the cigar string (sum of M + I)